<a href="https://colab.research.google.com/github/RoboTuan/ML4IOT_HMW/blob/main/Ex1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Tue Dec 15 08:00:58 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import argparse
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import tensorflow_model_optimization as tfmot
import tensorflow.lite as tflite
import zlib
import sys

In [3]:



#parser = argparse.ArgumentParser()
#parser.add_argument('--model', type=str, required=True, help='model name')
#parser.add_argument('--labels', type=int, required=True, help='model output')
#args = parser.parse_args()

seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

zip_path = tf.keras.utils.get_file(
    origin='https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip',
    fname='jena_climate_2009_2016.csv.zip',
    extract=True,
    cache_dir='.', cache_subdir='data')
csv_path, _ = os.path.splitext(zip_path)
df = pd.read_csv(csv_path)

column_indices = [2, 5]
columns = df.columns[column_indices]
data = df[columns].values.astype(np.float32)

n = len(data)
train_data = data[0:int(n*0.7)]
val_data = data[int(n*0.7):int(n*0.9)]
test_data = data[int(n*0.9):]

mean = train_data.mean(axis=0)
std = train_data.std(axis=0)

input_width = 6
LABEL_OPTIONS = 2


13574144/13568290 [==============================] - 1s 0us/step


In [4]:
x = np.array([1,2,3,4,5,6])
print(x[-1])
print(x[:-3])
print(x[-3:])

6
[1 2 3]
[4 5 6]


In [5]:
class WindowGenerator:
    def __init__(self, input_width, label_options, mean, std):
        self.input_width = input_width
        self.label_options = label_options
        self.mean = tf.reshape(tf.convert_to_tensor(mean), [1, 1, 2])
        self.std = tf.reshape(tf.convert_to_tensor(std), [1, 1, 2])

    def split_window(self, features):
        input_indeces = np.arange(self.input_width)
        inputs = features[:, :-6, :]
            
        labels = features[:, -6:, :]
        num_labels = 2

        inputs.set_shape([None, self.input_width, 2])
        # vedere se funge
        labels.set_shape([None, self.input_width, num_labels])

        return inputs, labels

    def normalize(self, features):
        features = (features - self.mean) / (self.std + 1.e-6)

        return features

    def preprocess(self, features):
        inputs, labels = self.split_window(features)
        inputs = self.normalize(inputs)

        return inputs, labels

    def make_dataset(self, data, train):
        ds = tf.keras.preprocessing.timeseries_dataset_from_array(
                data=data,
                # Targets None because we have the targets incorporated in the dataset
                targets=None,
                sequence_length=input_width+6,
                sequence_stride=1,
                batch_size=32)
        ds = ds.map(self.preprocess)
        ds = ds.cache()
        if train is True:
            ds = ds.shuffle(100, reshuffle_each_iteration=True)

        return ds


In [6]:
generator = WindowGenerator(input_width, LABEL_OPTIONS, mean, std)
train_ds = generator.make_dataset(train_data, True)
val_ds = generator.make_dataset(val_data, False)
test_ds = generator.make_dataset(test_data, False)

In [7]:
for element in train_ds:
    print(len(element))
    print(element[0].shape, element[1].shape)
    #print(element[0])
    sys.exit()

2
(32, 6, 2) (32, 6, 2)


SystemExit: 

## Models

In [8]:
MODEL_OPTIONS = "CNN-1D"
ALPHA = 1

units=6*2


if MODEL_OPTIONS == "MLP":
    model = keras.Sequential([
    keras.layers.Flatten(input_shape=(6, 2)),
    keras.layers.Dense(units=int(128*ALPHA), activation='relu', name='first_dense'),
    keras.layers.Dense(units=int(128*ALPHA), activation='relu', name='second_dense'),
    keras.layers.Dense(units, name='third_dense'),
    keras.layers.Reshape([6, 2])
    ])


elif MODEL_OPTIONS == "CNN-1D":
    model = keras.Sequential([
        keras.layers.Conv1D(filters=int(64*ALPHA), kernel_size=3, activation='relu', name='first_conv'),
        keras.layers.Flatten(input_shape=(64,)),
        keras.layers.Dense(units=int(64*ALPHA), activation='relu', name='first_dense'),
        keras.layers.Dense(units, name='second_dense'),
        keras.layers.Reshape([6, 2])
    ])

# WARNING: DO NOT USE LSTM MODEL
# NOT STABLE WITH TFLITE
# elif MODEL_OPTIONS == "LSTM":
#     model = keras.Sequential([
#         keras.layers.LSTM(64),
#         keras.layers.Flatten(),
#         keras.layers.Dense(units),
#         keras.layers.Reshape([6, 2])
#     ])

else:
    print("Invalid model selected")
    sys.exit()


In [9]:
# TODO: 
# pass a single list of dimension to the reduce_mean function
# see pdf for more instructions
class MsMoMAE(tf.keras.metrics.Metric):
    def __init__(self, name='MsMoMAE', **kwargs):
        super(MsMoMAE, self).__init__(name=name, **kwargs)
        self.total = self.add_weight(name='total', initializer='zeros', shape=(2, ))
        self.count = self.add_weight(name='count', initializer='zeros')

    def reset_states(self):
        self.count.assign(tf.zeros_like(self.count))
        self.total.assign(tf.zeros_like(self.total))

        return

    def update_state(self, y_true, y_pred, sample_weight=None):
        error = tf.abs(y_pred - y_true)
        #error = tf.reduce_mean(error, axis=1)
        #error = tf.reduce_mean(error, axis=0)
        error = tf.reduce_mean(error, axis=[0,1])
        self.total.assign_add(error)
        self.count.assign_add(1)

        return

    def result(self):
        result = tf.math.divide_no_nan(self.total, self.count)

        return result


In [10]:
model.compile(optimizer='adam',
            loss=[tf.keras.losses.MeanSquaredError()],
            metrics=[MsMoMAE()])

print("Fit model on training data")

history = model.fit(
    train_ds,
    batch_size=32,
    epochs=20,
    validation_data=(val_ds)
)

print("Evaluate on test data")
loss, error = model.evaluate(test_ds, verbose=2)
print(error)

model.summary()

Fit model on training data
Epoch 1/20
9200/9200 [==============================] - 54s 6ms/step - loss: 49.3406 - MsMoMAE: 2.6239 - val_loss: 4.9295 - val_MsMoMAE: 1.2850
Epoch 2/20
9200/9200 [==============================] - 31s 3ms/step - loss: 5.1622 - MsMoMAE: 1.3254 - val_loss: 5.8254 - val_MsMoMAE: 1.5140
Epoch 3/20
9200/9200 [==============================] - 31s 3ms/step - loss: 4.8162 - MsMoMAE: 1.2669 - val_loss: 5.5697 - val_MsMoMAE: 1.4541
Epoch 4/20
9200/9200 [==============================] - 32s 3ms/step - loss: 4.7534 - MsMoMAE: 1.2528 - val_loss: 5.8837 - val_MsMoMAE: 1.4134
Epoch 5/20
9200/9200 [==============================] - 31s 3ms/step - loss: 4.6003 - MsMoMAE: 1.2159 - val_loss: 4.9118 - val_MsMoMAE: 1.2966
Epoch 6/20
9200/9200 [==============================] - 31s 3ms/step - loss: 4.5110 - MsMoMAE: 1.1978 - val_loss: 5.6140 - val_MsMoMAE: 1.5401
Epoch 7/20
9200/9200 [==============================] - 31s 3ms/step - loss: 4.4321 - MsMoMAE: 1.1864 - val_loss: 

In [11]:
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude


pruning_params = {'pruning_schedule':
                tfmot.sparsity.keras.PolynomialDecay(
                    initial_sparsity=0.30,
                    final_sparsity=0.8,
                    begin_step=len(train_ds)*5,
                    end_step=len(train_ds)*15)}


model_for_pruning = prune_low_magnitude(model)


model_for_pruning.compile(optimizer='adam',
            loss=[tf.keras.losses.MeanSquaredError()],
            metrics=[MsMoMAE()])

model.summary()


callbacks = [
tfmot.sparsity.keras.UpdatePruningStep(),
#tfmot.sparsity.keras.PruningSummaries(log_dir=log_dir)
]

print("Fine tune pruned model")

history = model.fit(
    train_ds,
    batch_size=32,
    epochs=2,
    validation_data=(val_ds)
)

print("Evaluate on test data")
loss, error = model.evaluate(test_ds, verbose=2)
print(error)

model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)



Instructions for updating:
Please use `layer.add_weight` method instead.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
first_conv (Conv1D)          (None, 4, 64)             448       
_________________________________________________________________
flatten (Flatten)            (None, 256)               0         
_________________________________________________________________
first_dense (Dense)          (None, 64)                16448     
_________________________________________________________________
second_dense (Dense)         (None, 12)                780       
_________________________________________________________________
reshape (Reshape)            (None, 6, 2)              0         
Total params: 17,676
Trainable params: 17,676
Non-trainable params: 0
_________________________________________________________________
Fine tune pruned model
Epoch 1/2
9200/9200 [=

In [12]:
PTQ = "weights_activations"
MAGPRUNE = True

if MAGPRUNE == True:
    model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)
    final_model = model_for_export
else:
    final_model = model

saved_model_dir = './models/climate_{}_{}'.format(MODEL_OPTIONS, str(LABEL_OPTIONS))

if not os.path.exists('./models'):
    os.makedirs('./models')

if not os.path.exists(saved_model_dir):
    os.makedirs(saved_model_dir)
final_model.save(saved_model_dir)



converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model_dir = './models/tflite_climate_{}_{}_{}_{}_{}'.format(MODEL_OPTIONS, str(LABEL_OPTIONS), PTQ, str(ALPHA), str(MAGPRUNE))


if PTQ.lower() == 'weights':
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    tflite_model = converter.convert()

elif PTQ.lower() == 'weights_activations':

    def representative_dataset_gen():
        for x, _ in train_ds.take(1000):
            yield [x]

    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = representative_dataset_gen
    tflite_model = converter.convert()

else:
    tflite_model = converter.convert()

if MAGPRUNE == True:
    tflite_model_dir = tflite_model_dir + ".zip"
    with open(tflite_model_dir, 'wb') as fp:
        tflite_compressed = zlib.compress(tflite_model)
        fp.write(tflite_compressed)
    print(f"Size of tflite model: {os.path.getsize(tflite_model_dir)} bytes")
    
else:
    with open(tflite_model_dir, 'wb') as fp:
        fp.write(tflite_model)
    print(f"Size of tflite model: {os.path.getsize(tflite_model_dir)} bytes")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./models/climate_CNN-1D_2/assets
Size of tflite model: 14570 bytes


In [13]:
#!rm -rf ./models

In [ ]:
print(f"Size of tflite model: {os.path.getsize('./models/tflite_climate_CNN-1D_2_noPTQ_1')} bytes")
print(f"Size of tflite model: {os.path.getsize('./models/tflite_climate_CNN-1D_2_weights_1')} bytes")
print(f"Size of tflite model: {os.path.getsize('./models/tflite_climate_CNN-1D_2_weights_activations_1')} bytes")

Size of tflite model: 75740 bytes
Size of tflite model: 26848 bytes
Size of tflite model: 26320 bytes


In [22]:
str_object1 = open('./models/tflite_climate_CNN-1D_2_weights_activations_1_True.zip', 'rb').read()
str_object2 = zlib.decompress(str_object1)
f = open('./models/tflite_climate_CNN-1D_2_weights_activations_1_True', 'wb')
f.write(str_object2)
f.close()

interpreter = tflite.Interpreter(model_path="./models/tflite_climate_CNN-1D_2_weights_activations_1_True")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
